# EDA and Feature Engineering - Noise Events Prediction Model

This notebook will be used to explore the available data, where possible features for the predictive model will be evaluated.

In [1]:
import polars as pl
import numpy as np

## Meteo Data Exploration

In [2]:
df_meteo = pl.read_parquet('meteo_data.parquet')
df_meteo.head()

DATEUTC,ID,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Date,Year,Month,Day,Hour,Minute,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
str,str,i64,f64,i64,i64,f64,f64,i64,f64,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64
"""2022-01-01 00:…","""LC-002""",92,11.78,38,0,0.0,0.0,-169,0.43,"""2022-01-01""",2022,1,1,0,10,0,13.11,13.11,13.0515,13.048027
"""2022-01-01 00:…","""LC-002""",92,11.73,37,0,0.0,0.0,-170,0.33,"""2022-01-01""",2022,1,1,0,20,0,13.01,13.01,12.9515,12.985849
"""2022-01-01 00:…","""LC-002""",92,11.73,38,0,0.0,0.0,-167,0.46,"""2022-01-01""",2022,1,1,0,30,0,13.0,13.0,12.9415,12.950322
"""2022-01-01 00:…","""LC-002""",92,11.72,37,0,0.0,0.0,-160,0.52,"""2022-01-01""",2022,1,1,0,40,0,13.0,13.0,12.9415,12.94955
"""2022-01-01 00:…","""LC-002""",92,11.72,38,0,0.0,0.0,-166,0.51,"""2022-01-01""",2022,1,1,0,50,0,13.0,13.0,12.9415,12.952268


## Noise Events Exploration

In [3]:
df_events = pl.read_parquet('noise_events.parquet')
df_events.head()

#object_id,description,result_timestamp,noise_event_laeq_model_id,noise_event_laeq_model_id_unit,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_certainty_unit,noise_event_laeq_primary_detected_class,noise_event_laeq_primary_detected_class_unit
i64,str,str,i64,str,i64,str,str,str
255439,"""MP 01: Naamses…","""28/02/2022 08:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 13:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 16:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 19:…",null,null,null,"""%""",null,null
255439,"""MP 01: Naamses…","""28/02/2022 20:…",null,null,null,"""%""",null,null
